# Kaggle Walmart

In [ ]:
# Main packages
import numpy as np
import pandas as pd

#package sckit learn
#from sklearn.model_selection import train_test_split, RandomizedSearchCV
#from sklearn.linear_model import LogisticRegression

#from sklearn.metrics import mean_squared_error

#algo
#from xgboost import XGBClassifier

#Graph
from itertools import cycle
import matplotlib.pyplot as plt

#Memory usage 
import gc

color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
# loadin data

DIR='/kaggle/input/m5-forecasting-accuracy'
cal_df = pd.read_csv(f'{DIR}/calendar.csv')
sale_df = pd.read_csv(f'{DIR}/sales_train_validation.csv')
price_df = pd.read_csv(f'{DIR}/sell_prices.csv')
submission_df = pd.read_csv(f'{DIR}/sample_submission.csv')

In [ ]:
#Pour faire simple, tableau qui rassemble les informations sur les évenments et leur date dans les magasins et durant l'année. 
cal_df.sample(2)

In [ ]:
#Tableau rassemblant les ventes de tous les items en fonction des jours de ventes et des magasins 
sale_df.sample(2)


In [ ]:
x=1913/365
print("l'étude de vente dans les magasins s'étend sur 1913 jours soit {'x'} ans",x)

In [ ]:
#Le tableau rassemble les inforamtions sur les prix des différents items 
price_df.sample(2)

# Qu'es ce qu'on essaye de prédire ? 

Nous essayons de prévoir les ventes pour 28 jours de prévision. 

Les colonnes représentent 28 jours de prévision. Nous remplirons ces jours avec les prévisions.
Les lignes représentent chacune un article spécifique. Cet identifiant nous indique le type d'article, l'état et le magasin (et son numéro)



In [ ]:
submission_df.head()

# Ajustement de la mémoire des tableaux 
- Différentes variables sont enregistrées en int64 alors que in16 ou 8 est suffisant, donc nous changeons le types des variables lorsque c'est nécessaire

- Nous changeons aussi le type des variable 'object' en 'category', lorsque la variable se répète. 



In [ ]:
cal_df.info()

In [ ]:
##Ajustement de tableau Calendar.


#Pour réduire la consommation de RAM 
#Reduction de int64 a int8 ou int16 lorsque c'est nécessaire sur 

cal_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]] = cal_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]].astype("int8")
cal_df[["wm_yr_wk", "year"]] = cal_df[["wm_yr_wk", "year"]].astype("int16") 
cal_df["date"] = cal_df["date"].astype("datetime64")
#Suppréssion des valeurs NA
nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    cal_df[feature].fillna('unknown', inplace = True)

cal_df[["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] = cal_df[["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] .astype("category")

In [ ]:
#int16 sur le tableau sale
sale_df.loc[:, "d_1":] = sale_df.loc[:, "d_1":].astype("int16")

In [ ]:


#Réduction de la mémoire dans le tableau price 
price_df['sell_price']=price_df['sell_price'].astype("float16")
price_df['wm_yr_wk']=price_df['wm_yr_wk'].astype("int16")


# Regardons ce qu'il y a dans le tableau Sales

Le tableau Sale comprend l'id complet de l'item et le nombre de ventes par jours

In [ ]:
sale_df.head(2)

Nous représentons la vente d'un ITEM en fonction des ventes journalières

In [ ]:
d_cols = [c for c in sale_df.columns if 'd_' in c] # sales data colonne

# Différente étape 
# 1. Selectionner un item
# 2. Mettre l'id en index
# 3. Transformaer en colonne
# 4. Plot les données
sale_df.loc[sale_df['id'] == 'FOODS_1_001_CA_1_validation'] \
    .set_index('id')[d_cols] \
    .T \
    .plot(figsize=(15, 5),
          title='FOODS_1_001_CA_1 sales by "d" number')
plt.legend('')
plt.show()

# Fusion des Tableaux 'Calender' et 'Sale'

In [ ]:
sale_df

In [ ]:
exemple = sale_df.loc[sale_df['cat_id'] == 'HOBBIES']
exemple


In [ ]:
#Charge un Tableau avec les ventes de l'item choisis

exemple = sale_df.loc[sale_df['id'] == 'FOODS_1_001_CA_1_validation'][d_cols].T 
exemple = exemple.rename(columns={1612:'FOODS_3_090_CA_3'}) # Renomer correctement la colonne créer
exemple = exemple.reset_index().rename(columns={'index': 'd'}) # mettre 'd' en index
exemple = exemple.merge(cal_df, how='left', validate='1:1') #Merge le tableau sale et cal avec les indexs


exemple2 = sale_df.loc[sale_df['id'] == 'HOBBIES_1_001_CA_1_validation'][d_cols].T 
exemple2 = exemple2.rename(columns={0:'HOBBIES_1_001_CA_1'}) # Renomer correctement la colonne créer
exemple2 = exemple2.reset_index().rename(columns={'index': 'd'}) # mettre 'd' en index
exemple2 = exemple2.merge(cal_df, how='left', validate='1:1') #Merge le tableau sale et cal avec les indexs

exemple3 = sale_df.loc[sale_df['id'] == 'HOUSEHOLD_1_002_CA_1_validation'][d_cols].T 
exemple3 = exemple3.rename(columns={566:'HOUSEHOLD_1_002_CA_1'}) # Renomer correctement la colonne créer
exemple3 = exemple3.reset_index().rename(columns={'index': 'd'}) # mettre 'd' en index
exemple3 = exemple3.merge(cal_df, how='left', validate='1:1') #Merge le tableau sale et cal avec les indexs


exemple.set_index('date')['FOODS_3_090_CA_3'] \
    .plot(figsize=(15, 5),
          title='FOODS_3_090_CA_3 vente en focntion des dates')
plt.show()


In [ ]:
exemples = ['FOODS_3_090_CA_3','HOBBIES_1_001_CA_1','HOUSEHOLD_1_002_CA_1']
exemple_df = [exemple, exemple2, exemple3]
for i in [0, 1, 2]:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 3))
    exemple_df[i].groupby('wday').mean()[exemples[i]]\
        .plot(kind='line',
              title='average sale: day of week',
              lw=5,
              color='red',
              ax=ax1)
    exemple_df[i].groupby('month').mean()[exemples[i]] \
        .plot(kind='line',
              title='average sale: month',
              lw=5,
              color='blue',
              ax=ax2)
    exemple_df[i].groupby('year').mean()[exemples[i]] \
        .plot(kind='line',
              lw=5,
              title='average sale: year',
              color='orange',
              ax=ax3)
    fig.suptitle(f'Tendance pour l item: {exemples[i]}',
                 size=20,
                 y=1.1)
    plt.tight_layout()
    plt.show()

Nous remarquons que les ventes quotidiennes fluctuent beacuoup pour pouvoir les prédir en fonction des jours

Les années nous montrent déja une légère tendance.


# Présentation d'item en fonction du temps

Présentons 20 articles choisis aux hasards en fonction du temps, et observons les différences.
- Il est fréquent de voir un élément indisponible pendant un certain temps.
Certains articles ne se vendent qu'une fois ou moins par jour, ce qui rend la situation très difficile à prévoir.

- D'autres articles affichent des pics de demande (evnmt aux USA), nous regarderons la colonne événements plus tard.



In [ ]:
twenty_examples = sale_df.sample(20, random_state=529) \
        .set_index('id')[d_cols] \
    .T \
    .merge(cal_df.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(15, 20))
axs = axs.flatten()
ax_idx = 0
for item in twenty_examples.columns:
    twenty_examples[item].plot(title=item,
                              color=next(color_cycle),
                              ax=axs[ax_idx])
    ax_idx += 1
plt.tight_layout()
plt.show()

# Fusion des Tableaux

**Qu'es ce qu'on cherche a mettre en index ? **
- On cherche à mettre en index les dates pour pouvoir prédir les 30 jours suivants (F_1,F_2,...,F_28)

**Qu'es ce qu'on cherche à mettre en avant ?**
- La date, et les évenements sont des varibeles qui influencent beacoup sur les ventes.
- Les prix sont évidament qlq choses d'important, il faudra les travailler de façons intéligent.


**Qu'es ce qu'on peut supprimer? Colonne inutile dans un premier temps ?**
- Pour l'instant aucune colonne me parait inutile, a voir qu'es ce que donne les snap.



Le tableau price et sale peut être fusioner sur la clé id après avoir ajuster le tebleau price 

In [ ]:
#Ajout de L'id complet dans le tableau price
price_df.loc[:, "id"] = price_df.loc[:, "item_id"] + "_" + price_df.loc[:, "store_id"] + "_validation"

On peut donc fusioner le tableau price et sale ensemble sur la clé unique 'id'

In [ ]:

def make_dataframe():
    df = pd.merge(price_df, sale_df, on="id")
    return df

In [ ]:
df1=make_dataframe()
df1.head()

Desormais il reste à fusioner le tableau créée et le tableau calender
<br>
Regardons sur quel clé peut on les fusioner 
<br>
La seul clé unique est la variable' d_' présentent dans les deux tableau mais il faudra faire des modifications

In [ ]:
cal_df.head(5)

Le deux tableau ont en commun la varianle 'd_' qui sont les ventes du jour
- d_1 correspond au vente 2011-01-29	
- d_XX correspond a la dernière vente enregistré (j'ai pas la dernière valeur puisque je travail avec un subset du tableau pour l'instant)

Ce qu'il faut faire :
- Transposer toutes les lignes d_1 jusqu'à d_xx et ainsi créer la clé unique 


Nous partons d'une feuille blanche, c'est a dire
- Etape 1 : prendre la tableau 'sale', et supprimer toute les colonnes non voulues et transoser les colonnes 'd_'
- Etape 2 : ajouter en index la date du tableau calender qui correspond au vente de la variable 'd_' dans le tableau sale.
- Etape 3 : Ajout des 'id' des items

In [ ]:
#creation d'un tableau des ventes d'un item en fonction du jour de vente
df_id= sale_df.drop(columns=["item_id", "dept_id", "cat_id", "state_id","store_id", "id"]).T
df_id.index = cal_df["date"][:1913]
df_id.columns = sale_df["id"]


In [ ]:
df_id.head()

C'est un bon départ mais il faut que les noms des items soient en colonnes avec leurs nombres de ventes associé en colonnes.

In [ ]:
df_long = df_id.stack().reset_index(1)#.Sack permet de les mettres en colonnes // et .reset_index(1) permet de de mettre la colonne date dans une colonne variable (plus en index ! tu as compris ????)

df_long.columns = ["id", "value"]#rename des colonnes

In [ ]:
df_long

Desormais nous avons les clé uniqeu id et date qui permet de fusioner les 3 tableaux 

In [ ]:
cal_df.head(2)

In [ ]:
df = pd.merge(df_long.reset_index(), cal_df, on='date')

In [ ]:
df= pd.merge(df, price_df, on=["id", "wm_yr_wk"])
df

In [ ]:
df.info()

Finalement nous avons utlisé une autre technique

-Nous créons un tableau avec les variables "date", "id", "value"(nombre d'item vendu en fonction de la date).

-Puis nous fusionons les tableaux 'price' et 'calendar'.

    Vous avez bien remarquez que nous avons pas fuisonner le tableau "sale_df' (ci-dessous), il nous manque donc les variable :
        - 'dept_id", 'dept_id', store_id et 'state_id'
    Nous allons donc les céer à partir de l'id de l'item, présant dans le tableau 'price' par exemple.
   

In [ ]:
price_df.head(2)

In [ ]:
price_df = pd.concat([price_df, price_df["item_id"].str.split("_", expand=True)], axis=1)#création des variables 'cat_id' et 'dept_id'
price_df = price_df.rename(columns={0:"cat_id", 1:"dept_id"})#Rename
price_df[["store_id", "item_id", "cat_id", "dept_id"]] = price_df[["store_id","item_id", "cat_id", "dept_id"]].astype("category")#Ajustement Memory Usage
price_df = price_df.drop(columns=2)#Drop colonne inutile

In [ ]:
price_df.head(2)

In [ ]:
def make_dataframe():

    df_id = sale_df.drop(columns=["item_id", "dept_id", "cat_id", "state_id","store_id", "id"]).T
    df_id.index = cal_df["date"][:1913]
    df_id.columns = sale_df["id"]
    



    df_long = df_id.stack().reset_index(1)
    df_long.columns = ["id", "value"]

    del df_id
    gc.collect()
    
    df = pd.merge(pd.merge(df_long.reset_index(), cal_df, on="date"), price_df, on=["id", "wm_yr_wk"])
    df = df.drop(columns=["d"])
#     df[["cat_id", "store_id", "item_id", "id", "dept_id"]] = df[["cat_id"", store_id", "item_id", "id", "dept_id"]].astype("category")


    del df_long
    gc.collect()

    return df


In [ ]:
df = make_dataframe()
df.head()

In [ ]:
df.info()

In [ ]:
df